In [86]:
!pip install ollama
!pip install langchain chromadb gradio 
!pip install -U langchain-community
!pip install pymupdf

In [87]:
# Import necessary packages
import ollama # Ollama API
import gradio as gr # Interface

# Document processing  
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Embedding generation  
from langchain_community.embeddings import OllamaEmbeddings

import re

In [88]:
# Question au chat bot
response = ollama.chat(
    model="deepseek-r1:1.5b",
    messages=[
        {"role": "user", "content": "Explain Newton's second law of motion"},
    ],
)

print(response["message"]["content"])

<think>
Okay, so I need to explain Newton's Second Law of Motion. Hmm, let me think about what I remember from my physics class.

Newton's laws are all about how things move and change speed or direction when forces act on them. The first one was about if you have no net force, the object stays at rest or moves at a constant velocity. That makes sense because gravity is pulling everything down, so things fall to Earth.

The second law must be more significant. I think it's F equals mass times acceleration (F=ma). So when there's an unbalanced force acting on something, its acceleration changes. That sounds right. But why does that matter? Well, if you push a box harder, it accelerates faster. If the force is bigger or the mass of the box is smaller, even with the same force, it moves slower.

Wait, but what about when the object isn't moving or not changing direction? Oh, right, if there's no acceleration (because F=0), then a=0. That makes sense because if you stop pushing, the object

In [89]:
def process_pdf(pdf_bytes):
    
    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_documents(data)

    # Transphorme texte en vecteurs
    embeddings = OllamaEmbeddings(model="bge-m3")
    # Crée la bd vectorielle
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="./chroma_db")
    # Object pour la recherche
    retriever = vectorstore.as_retriever()
    return text_splitter, vectorstore, retriever

In [90]:
text_splitter, vectorstore, retriever = process_pdf("Rapport Alternance S6 - Téo Viglietti.pdf")

In [122]:
# Test de la recherche
print("Recherche dans la base de données vectorielle...")
docs = retriever.get_relevant_documents("Application de suivi des tâches des RDG",k=4)
for doc in docs:
    print(doc.metadata)
    print(doc.page_content)

Recherche dans la base de données vectorielle...
{'moddate': '2025-06-23T00:36:02-07:00', 'producer': '', 'total_pages': 35, 'format': 'PDF 1.7', 'author': 'admin', 'creationdate': '2025-06-23T00:36:02-07:00', 'creationDate': "D:20250623003602-07'00'", 'subject': '', 'modDate': "D:20250623003602-07'00'", 'keywords': '', 'page': 16, 'source': 'Rapport Alternance S6 - Téo Viglietti.pdf', 'trapped': '', 'creator': 'Microsoft Word', 'file_path': 'Rapport Alternance S6 - Téo Viglietti.pdf', 'title': ''}
RAPPORT FINAL – ECUE ALTERNANCE S6 
 
 
 
 
   
 
16 | P A G E  
 
tâches effectuées par les RDG. Ces tâches sont codifiées et transmises via le remplissage d’une feuille de 
service. 
 
Application de suivi des tâches des RDG 
Dans ce cadre, j’ai été chargé par M. Pierre Bolot et M. Stéphane Piozza de modifier le rapport « Tableau 
de bord RDS » ainsi que de créer sa publication dans SAP BO. L’objectif était d’ajouter deux onglets
{'moddate': '2025-06-23T00:36:02-07:00', 'producer': '', 'au

In [119]:
def combine_docs(docs):
    l_docs = []
    for doc in docs:
        if doc.page_content not in l_docs:
            l_docs.append(doc.page_content)
    return "\n\n".join(l_docs)

In [120]:
combine_docs(docs)

'RAPPORT FINAL – ECUE ALTERNANCE S6 \n \n \n \n \n   \n \n16 | P A G E  \n \ntâches effectuées par les RDG. Ces tâches sont codifiées et transmises via le remplissage d’une feuille de \nservice. \n \nApplication de suivi des tâches des RDG \nDans ce cadre, j’ai été chargé par M. Pierre Bolot et M. Stéphane Piozza de modifier le rapport « Tableau \nde bord RDS » ainsi que de créer sa publication dans SAP BO. L’objectif était d’ajouter deux onglets'

In [126]:
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}\n\n Répond uniquement en français."
    response = ollama.chat(
        model="gemma3n:e2b",
        messages=[{'role': 'user', 'content': formatted_prompt}]
    )

    response_content = response['message']['content']
    final_answer = re.sub(r'<think>.*?</think>',
                          '',
                          response_content,
                          flags=re.DOTALL).strip()
    return final_answer

In [127]:
question = "Application de suivi des tâches des RDG"
docs = retriever.get_relevant_documents(question,k=4)
combined_text = combine_docs(docs)
print(combined_text)
ollama_llm(question, combined_text)

RAPPORT FINAL – ECUE ALTERNANCE S6 
 
 
 
 
   
 
16 | P A G E  
 
tâches effectuées par les RDG. Ces tâches sont codifiées et transmises via le remplissage d’une feuille de 
service. 
 
Application de suivi des tâches des RDG 
Dans ce cadre, j’ai été chargé par M. Pierre Bolot et M. Stéphane Piozza de modifier le rapport « Tableau 
de bord RDS » ainsi que de créer sa publication dans SAP BO. L’objectif était d’ajouter deux onglets


'L\'application de suivi des tâches des RDG est un système où les tâches effectuées par les RDG sont codifiées et transmises via un formulaire de service.\n\nL\'objectif de la modification du rapport "Tableau de bord RDS" et de sa publication dans SAP BO, comme vous l\'avez mentionné, était d\'ajouter deux onglets.'

In [132]:
import gradio as gr

def ask_model(question):
    docs = retriever.get_relevant_documents(question, k=4)
    combined_text = combine_docs(docs)
    response = ollama_llm(question, combined_text)
    return response

# Création de l'interface
interface = gr.Interface(
    fn=ask_model,
    inputs=gr.Textbox(lines=2, label="Pose ta question"),
    outputs=gr.Textbox(label="Réponse du modèle"),
    title="Interface avec Ollama",
    description="Pose une question"
)

# Lancement
interface.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
